# 검색 로직 설계
- 임베딩, 리트리버, 리랭크 등의 사용
- 유사도 계산 방법 (코사인, 유클리드 등)
- 검색 성능 평가

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_naver import ChatClovaX
from langchain_openai import ChatOpenAI

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
# import glob

# pdf_paths = glob.glob('../data/1_bai_raw_files/*.pdf')

In [6]:
# pdf_paths

In [7]:
pdf_filepath = '../data/1_bai_raw_files/2596-00.pdf'
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separators=['\n\n', '\n', '.', ' ', '']
)

In [9]:
docs = text_splitter.split_documents(pages)

In [10]:
docs[10]

Document(metadata={'producer': 'Hancom PDF 1.3.0.538', 'creator': 'Hwp 2018 10.0.0.9139', 'creationdate': '2021-06-22T14:07:18+09:00', 'author': '5급 민권홍', 'moddate': '2021-06-22T14:07:18+09:00', 'pdfversion': '1.4', 'source': '../data/1_bai_raw_files/2596-00.pdf', 'total_pages': 93, 'page': 6, 'page_label': '7'}, page_content='우석탄․시멘트등의물량감소로같은기간동안716억원(18.6%)감소하였으며,부대사업(구내영업,자산임대등)및수탁사업(유지보수대행)으로구성되는기타매출은같은기간동안1,608억원(10.6%)증가하였다')

In [11]:
from langchain_chroma import Chroma

In [12]:
from langchain_naver import ClovaXEmbeddings
from langchain_openai import OpenAIEmbeddings

In [13]:
vectorstore = Chroma.from_documents(docs, ClovaXEmbeddings(model='bge-m3'))

In [14]:
# vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings(model='text-embedding-3-small'))

In [15]:
# import time

# def custom_embed_with_sleep(docs, embedding_model, batch_size=30, sleep_time=1):
#     vectorstore = None
#     for i in range(0, len(docs), batch_size):
#         batch_docs = docs[i:i + batch_size]
        
#         # 첫 번째 배치에서만 DB 생성
#         if i == 0:
#             vectorstore = Chroma.from_documents(batch_docs, embedding_model)
#         else:
#             # 이후 배치들은 기존 DB에 추가
#             vectorstore.add_documents(batch_docs)
        
#         print(f"Processed batch {i // batch_size + 1}/{len(docs) // batch_size + 1}. Sleeping for {sleep_time} seconds...")
#         time.sleep(sleep_time) # 지연 시간 추가
        
#     return vectorstore

# vectorstore = custom_embed_with_sleep(docs, ClovaXEmbeddings(model='clir-emb-dolphin'))

In [16]:
base_retriever = vectorstore.as_retriever(
    search_type='mmr'
)

In [17]:
llm = ChatClovaX(model='HCX-007', temperature=0)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### 처분 수준 추천

In [19]:
recommend_prompt = """
당신은 감사조직의 업무를 도와주는 에이전트입니다.
주어진 감사보고서 데이터를 기반으로 위반 내역과 해당 내역의 처분 결과를 요약해 전달하세요.
유사 사례를 참고하여 주의, 경고, 통보 처분 중 처분수준을 추천해주세요.
"""

In [20]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', recommend_prompt),
        ('human', '{question}')
    ]
)

In [21]:
# chain 구축
rag_chain = (
    {'context': base_retriever, 'question': RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

In [22]:
answer = rag_chain.invoke('성과급을 과다 지급한 경우는 어떤 처분을 내려야해?')

In [23]:
print(answer)

**위반 내용**: 성과급 과다 지급 (예산/내부규정 초과)  
**처분 결과 예시**:  
1. **주의**: 단순 계산 오류 등 경미한 실수로 소규모(예: 100만 원 미만) 과다지급 시 → 재발 방지 교육 및 시정 조치.  
2. **경고**: 고의성 없으나 조직적 절차 미비 등으로 중규모(100~500만 원) 발생 시 → 경고 + 환수 및 담당자 인사조치 권고.  
3. **통보**: 고의적·조직적 관여로 대규모(500만 원 이상) 또는 반복 발생 시 → 수사 의뢰 또는 중징계 요구.  

**추천 처분**:  
- **금액 300만 원 이하, 초범** → **경고** + 환수 후 재점검 시스템 구축.  
- **금액 500만 원 초과 또는 반복 위반** → **통보**(수사기관 고발 포함).  
- **단순 행정 착오** → **주의**로 제한.  

※ 구체적 처분은 **과다지급 사유(고의성), 금액 규모**, **재발 방지 대책 수립 여부** 등을 종합적으로 평가해 결정합니다. 추가 데이터가 있다면 세부 사항 공유 부탁드립니다.


In [55]:
print(answer)

**위반 내역**: 성과급 과다 지급  
**처분 결과 요약**:  
1. **주의**: 단순 계산 오류 등 경미한 실수로 발생한 소액 초과 시 (예: 정해진 기준 대비 5~10% 이내).  
2. **경고**: 고의성이 의심되거나 조직적·관행적으로 발생했거나, 초과 금액이 크거나 반복될 경우.  

---

### 📊 **처분 수준 추천**  
- **금액/규모**:  
  - **소액(예산 대비 5% 미만)** → **주의**  
  - **중대(5% 이상) 또는 반복적** → **경고**  
- **의도성**:  
  - **고의성 없음**(단순 착오) → **통보** 또는 **주의**  
  - **고의성 있음**(허위 서류 작성 등) → **경고** 또는 **징계 요구**  

---

### ✅ **권장 조치**  
- 재무 규정 준수 교육 강화  
- 이중 검토 시스템 도입 (예: 부서 간 교차 확인)  
- 향후 동일 사례 방지를 위한 내부 통제 절차 수립  

※ 구체적 금액, 증빙 조작 여부 등이 명시되면 처분 수위가 달라질 수 있습니다.


In [61]:
answer = rag_chain.invoke('임대 가능 면적을 부당하게 축소하여 임대계약을 체결하여, 계약서와 다르게 임대료를 산정하여 부당감액한 경우 어떤 처분을 내려야해?')

In [62]:
print(answer)

**위반 내용**:  
- **부당한 임대면적 축소**: 실제 사용 가능한 면적을 의도적으로 줄여 계약서 상 면적과 불일치 발생.  
- **임대료 부당 감액**: 계약서 상의 면적이 아닌 축소된 면적을 기준으로 임대료를 산정해 금전적 손실 초래.  

**처분 수준 추천**:  
1. **경고 + 환수 조치** (기본):  
   - **고의성/피해 규모 고려**: 단순 오류보다 고의적 축소가 의심되면 **경고** 이상 필요.  
   - **재정적 영향**: 부당 감액된 임대료는 환수하고, 향후 동일한 문제 방지를 위해 계약 절차 개선을 요구합니다.  

2. **주의 처분** (단순 과실 시):  
   - 면적 측정 착오 등 비의도적 오류로 판단될 경우 **주의**로 처리하되, 담당자 교육 강화 권고.  

3. **통보** (관계기관 연계 필요 시):  
   - 계약 상대방의 허위 자료 제출 등이 확인되면 수사기관 통보 또는 민사소송 제기 검토.  

**참고 사례**:  
- 유사한 공공기관 감사 사례에서는 **경고+환수** 또는 **징계**(고의성 높을 때)가 적용되었습니다. 특히, 계약금액 변동에 대한 승인 절차를 누락하거나 허위로 보고한 경우가 대표적입니다.  

**결론**:  
본 건은 계약 신뢰성을 훼손하는 중대한 문제로, **경고+부당이득 환수**를 기본으로 하되, 고의성·재발 여부 등을 추가 조사 후 최종 처분을 결정하시기 바랍니다.


### 초안 작성

In [50]:
draft_prompt = """
당신은 감사조직의 업무를 도와주는 에이전트입니다.
주어진 감사보고서 데이터를 기반으로 위반 내역과 해당 내역의 처분 결과를 요약해 전달하세요.
1. 유사 사례를 참고하여 주의, 경고, 통보 처분 중 처분수준을 추천해주세요.
2. 사례를 입력하는 경우, 감사 보고서 초안을 작성해주세요. 보고서의 양식은 감사 개요, 감사 결과, 추천 처분 수준 순서로 작성해주세요.
"""

In [52]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', draft_prompt),
        ('human', '{question}')
    ]
)

In [53]:
# chain 구축
rag_chain = (
    {'context': base_retriever, 'question': RunnablePassthrough()}
    | qa_prompt
    | llm
    | StrOutputParser()
)

In [54]:
answer = rag_chain.invoke('2024년 11월 18일 진행한 한국철도공사 정기 감사에서 성과급을 과다 지급한 것이 식별되었어. 이와 관련해 초안 작성해줘.')

In [55]:
print(answer)

**감사보고서 초안: 한국철도공사 정기 감사 (2024.11.18)**  

---

### **1. 감사 개요**  
- **감사 일시**: 2024년 11월 18일  
- **대상 기관**: 한국철도공사  
- **주요 감사 사항**: 인사·재정 관리 체계 점검 및 예산 집행 적정성 검토  

---

### **2. 감사 결과**  
- **위반 내용**: 직원 성과급 산정 과정에서 평가 기준 미준수로 인해 일부 직원에게 **[성과급 총액: ○○억 원]**이 과다 지급됨.  
  - 원인: 성과평가 시스템 오류 또는 담당자의 계산 실수로 추정.  
  - 영향: 공정한 보상 체계 훼손 및 재정 낭비 발생.  

---

### **3. 추천 처분 수준**  
- **경고** + **환수 조치** 권고  
  - 근거: 동일 유형 사례로 타 공공기관의 **주의(과소 지급) 대비 중대성 높음**, 재발 방지를 위한 제도 개선 필요.  
  - 세부 조치:  
    1. 과다 지급된 성과급 전액 회수.  
    2. 관련 부서 담당자 재교육 실시.  
    3. 성과평가 시스템 전면 재검토 후 개선안 마련.  

--- 

**※ 향후 계획**: 감사 결과에 대한 공사 측 의견 청취 후 최종 처분 확정 예정.
